## 基本概念

首先定义一个长度为 N 的输入序列为：
$$
\mathbb{S}_{N}=\{ w_i \}_{i=1}^{N} 
$$

其中 $w_i$ 表示输入序列中第 i 个 token，而输入序列 $\mathbb{S}_N$ 对应的 embedding 表示为：
$$
\mathbb{E}_{N}=\{ \bm{x}_i \}_{i=1}^N
$$

其中 $\bm{x}_i$ 表示第 i 个 token $w_i$ 对应的 d 维词嵌入向量。

接着在做 self-attention 之前，会用词嵌入向量计算 $\bm{q,k,v}$ 向量同时加入位置信息，函数公式表达如下：
$$
\bm{q}_m=f_q(\bm{x}_m,m) \\ 
\bm{k}_n=f_k(\bm{x}_n,n) \\ 
\bm{v}_n=f_v(\bm{x}_n,n)
$$

其中 $\bm{q}_m$ 表示第 m 个 token 对应的词向量 $\bm{x}_m$ 集成位置信息 m 之后的 query 向量。而 $\bm{k}_n$ 和 $\bm{v}_n$ 则表示第 n 个 token 对应的词向量 $\bm{x}_n$ 集成位置信息 n 之后的 key 和 value 向量。

而基于 transformer 的位置编码方法都是着重于构造一个合适的 $f\left( \bm{q},\bm{k},\bm{v} \right)$ 函数形式。

而计算第 m 个词嵌入向量 $\bm{x}_m$ 对应的 self-attention 输出结果，就是 $\bm{q}_m$ 和其他 $\bm{k}_n$ 都计算一个 attention score ，然后再将 attention score 乘以对应的 $\bm{v}_n$ 再求和得到输出向量 $\bm{o}_m$ ：
$$
a_{m,n}=\frac{\text{exp}(\frac{\bm{q}_m^{\textbf{T}}\bm{k}_n}{\sqrt{d}})}{\sum_{j=1}^N\text{exp}(\frac{\bm{q}_m^{\textbf{T}}\bm{k}_j}{\sqrt{d}})} \\ 
\bm{o}_m=\sum_{n=1}^Na_{m,n}\bm{v}_n
$$


## 绝对位置编码

对于位置编码，常规的做法是在计算 query, key 和 value 向量之前，会计算一个位置编码向量 $\bm p_i$ 加到词嵌入 $\bm x_i$ 上，位置编码向量 $\bm p_i$ 同样也是 d  维向量，然后再乘以对应的变换矩阵:
$$
\bm f_{t:t\in {q,k,v}}(\bm x_i, i) := \bm W_{t:t\in {q,k,v}}(\bm x_i + \bm p_i)
$$

而经典的位置编码向量 $\bm p_i$ 的计算方式是使用 Sinusoidal 函数：
$$
\bm P_{i, 2t} = \sin \left ( k/10000^{2t}/d \right ) \\
\bm P_{i, 2t+1} = \cos \left ( k/10000^{2t}/d \right )
$$

其中 $\bm P_{i, 2t}$ 表示位置 d 维度向量 $\bm p_i$ 中的第 $\bm p_i$ 位置分量也就是偶数索引位置的计算公式，而 $\bm P_{i, 2t+1}$就对应第 $2t+1$ 位置分量也就是奇数索引位置的计算公式。

## 旋转位置编码（Rotary Position Embedding，RoPE）

论文中提出为了能利用上 token 之间的相对位置信息，假定 query 向量 $\bm q_m$ 和 key 向量 $\bm k_n$ 之间的内积操作可以被一个函数 $\mathcal g$ 表示，该函数 $\mathcal g$ 的输入是词嵌入向量 $\bm x_m$，$\bm x_n$ 和它们之间的相对位置 $m - n$ ：
$$
\langle \bm f_q(\bm x_m, m), \bm f (\bm x_n, n) \rangle = g(\bm x_m, \bm x_n, m-n)
$$

接下来的目标就是找到一个等价的位置编码方式，从而使得上述关系成立。

假定现在词嵌入向量的维度是两维 $d=2$，这样就可以利用上2维度平面上的向量的几何性质，然后论文中提出了一个满足上述关系的 $f$ 和 $g$ 的形式如下：
$$
f_q(\bm x_m, m) = (\bm W_q \bm x_m)e^{im\theta} \\
f_k(\bm x_n, n) = (\bm W_k \bm x_n)e^{in\theta} \\
g(\bm x_m, \bm x_n, m-n) = Re\left[(\bm W_q\bm x_m)(\bm W_k\bm x_n)^*e^{i(m-n)\theta}\right]
$$

这里面 Re 表示复数的实部。

进一步地，$f_q$ 可以表示成下面的式子：

 

看到这里会发现，这不就是 query 向量乘以了一个旋转矩阵吗？这就是为什么叫做旋转位置编码的原因。

同理，$f_k$ 可以表示成下面的式子：

 

最终 $g(\bm x_m, \bm x_n, m-n)$ 可以表示如下：
